<a href="https://colab.research.google.com/github/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/Topic_Modeling_with_BERTopic_Reclame_aqui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Topic Modeling with BERTopic - Reclame Aqui**

BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions (https://maartengr.github.io/BERTopic/index.html).

### **Enabling the GPU**

We will use the GPU provided by COLAB to accelarate our model training. To enable GPUs for the notebook:
1- Navigate to Edit -> Notebook Settings
2- Select GPU from the Hardware Accelerator drop-down

In [ ]:
# verify if GPU is enable
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

### **Setup**

In [ ]:
import pandas as pd # for data manipulation
import os # for interacting with the operating system
import nltk # for natural language processing
import string # for string manipulation
import re # for for regular expressions
import matplotlib.pyplot as plt # for visualization
import spacy # for lemmatize portuguese text
try:
  from bertopic import BERTopic # for topic modeling
  from spellchecker import SpellChecker # for spell check
except:
  !pip install pyspellchecker
  !pip install bertopic
  !pip install kaleido # for save BERTopic plots as image
  #os.kill(os.getpid(), 9)

In [ ]:
# Install spacy pt_core_news_sm for portuguese text
!python -m spacy download pt_core_news_sm

In [ ]:
nltk.download("punkt")

In [ ]:
# Download dataset with stopwords
nltk.download("stopwords")

In [ ]:
# Donwload datasets for lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
# Donwload dependency need to stem portuguese text
nltk.download('rslp')

### **Load data from [Github](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui.git)**

In [ ]:
!git clone https://github.com/punkmic/Topic-Modeling-Reclame-Aqui.git

Cloning into 'Topic-Modeling-Reclame-Aqui'...
remote: Enumerating objects: 16139, done.
remote: Counting objects: 100% (1958/1958), done.
remote: Compressing objects: 100% (1759/1759), done.
remote: Total 16139 (delta 238), reused 1808 (delta 173), pack-reused 14181
Receiving objects: 100% (16139/16139), 118.28 MiB | 15.35 MiB/s, done.
Resolving deltas: 100% (1386/1386), done.
Checking out files: 100% (14795/14795), done.


In [ ]:
# Change directory
%cd /content/Topic-Modeling-Reclame-Aqui 

# Update files from remote repository
!git pull 

# Return to work directory
%cd ..

# Check current directory
!pwd

/content/Topic-Modeling-Reclame-Aqui
Already up to date.
/content
/content


## **Prepare data**

In [ ]:
def read_data(path_csv, drop_duplicates = True, lower=True):
 
  # Use the read_csv method to read csv file
  df = pd.read_csv(path_csv)
  
  if drop_duplicates:
    # Read and return the CSV file using the read_csv method
    print(f"Shape before remove duplicates: {df.shape}")

    # Use the drop_duplicated method to drop duplicates rows
    df = df.drop_duplicates(subset="text")

    print(f"Shape after remove duplicates: {df.shape}")

    if lower:
      # apply the str.lower() method to each element in the dataframe
      df = df.applymap(str.lower)
  return df

In [ ]:
# Set the path to data
path_csv = "/content/Topic-Modeling-Reclame-Aqui/corpus.csv"

df = read_data(path_csv)

# Print the first 5 rows of the DataFrame
df.head(5)

Shape before remove duplicates: (12760, 2)
Shape after remove duplicates: (10510, 2)


,title,text
0,pedido cancelado sem justificativa após uma se...,eu estava pesquisando bastante uma nova tv par...
1,pedido cancelado,eu sinceramente estou decepcionada com o amazo...
2,cobrança indevida,cancelei meu plano antes de terminar o período...
3,pedido reincidente,"olha fiz compra veio errada, e veio errado nov..."
4,assinatura para vender na amazon brasil,eu me inscrevi na amazon para realizar vendas ...


This dataset contains just two columns called title and text 

In [ ]:
print(df.shape)

(10510, 2)


There are 10510 unique rows in this dataset.

In [ ]:
# join columns
df["documents"] = df["title"] + " " + df["text"]

# Use the replace() method to replace the string with an empty string
df = df.replace(re.compile('\[editado pelo reclame aqui\]|editado pelo reclame aqui|Editado pelo Reclame Aqui'), '')

# Drop the old index column
df.reset_index(inplace = True, drop = True)

df.head()

,title,text,documents
0,pedido cancelado sem justificativa após uma se...,eu estava pesquisando bastante uma nova tv par...,pedido cancelado sem justificativa após uma se...
1,pedido cancelado,eu sinceramente estou decepcionada com o amazo...,pedido cancelado eu sinceramente estou decepc...
2,cobrança indevida,cancelei meu plano antes de terminar o período...,cobrança indevida cancelei meu plano antes de ...
3,pedido reincidente,"olha fiz compra veio errada, e veio errado nov...",pedido reincidente olha fiz compra veio errada...
4,assinatura para vender na amazon brasil,eu me inscrevi na amazon para realizar vendas ...,assinatura para vender na amazon brasil eu me ...


#### **Save table as image**

In [ ]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/joined_table/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)

# Save the DataFrame to a CSV file
df.head(10).to_csv(path + 'joined_table.csv')

### **Preprocessing**

#### **Tokenization**

Tokenization aims to breaking text down into its component parts

In [ ]:
WORD_TOKENIZER = nltk.tokenize.word_tokenize
def tokenize(text, lowercase=True):
  if lowercase:
    text = text.lower()
  return WORD_TOKENIZER(text, language="portuguese")

#### **Stem** 

Stem the tokens. This step aims to remove morphological affixes and normalize to standardized stem forms

In [ ]:
STEMMER = nltk.stem.RSLPStemmer()
def stem(tokens):
  return [STEMMER.stem(token) for token in tokens]

#### **Lemmatize**

Lemmatize the tokens. Retains more natural forms than stemming, but assumes all tokens nons unless tokens are passed as (word, pos) tuples. Note: nltk lemmatize does not suport portugues language

In [ ]:
LEMMATIZER = nltk.WordNetLemmatizer()

def lemmatize(tokens):
  lemmas = []
  for token in tokens:
      if isinstance(token, str):
          # treats token like a noun
          lemmas.append(LEMMATIZER.lemmatize(token)) 
      else: 
          # assume a tuple of (word, pos)
          lemmas.append(LEMMATIZER.lemmatize(*token))
  return lemmas

**Lemmatize option for portuguese text**

In [ ]:
def lemmatize_pt(tokens, nlp):
  # Create a spaCy Doc object and apply the lemmatization
  doc = nlp(' '.join(tokens))

  # Return lemmatize
  return [token.lemma_ for token in doc]

#### **Remove stopwords**

Stop words are things like articles and conjunctions that usually do not offer a lot of value in an analysis.

In [ ]:
def remove_stopwords(tokens, stopwords=None):

  # Use the default stop words if none is passed
  if stopwords is None:
    stopwords = nltk.corpus.stopwords.words('portuguese')
  
  # Filter the list of tokens to exclude the stop word tokens
  return [token for token in tokens if token not in stopwords]

#### **Remove hyperlinks**

Removes http/s links from the tokens.

In [ ]:
def remove_links(tokens):
  # Filter tokens that starts with "http://" or "https://"
  return [token for token in tokens 
          if not token.startswith("http://")
          and not token.startswith("https://")]

#### **Remove numbers**

In [ ]:
def remove_numbers(tokens):
  # Filter out number tokens using a list comprehension and the isnumeric method
  return [token for token in tokens if not token.isnumeric()]


#### **Remove date**

In [ ]:
def remove_date(tokens):
  # Compile a regular expression to match dates in the format dd/mm or dd/mm/yyyy
  date_regex = re.compile(r'\d{2}/\d{2}(/\d{4})?')

  # Use the regex to find all the tokens that match the date pattern
  dates = [token for token in tokens if date_regex.fullmatch(token)]

  # Filter the list of tokens to exclude the date tokens
  filtered_tokens = [token for token in tokens if token not in dates]

  # Return the filtered tokens
  return filtered_tokens

#### **Remove punctuation**

In [ ]:
def remove_punctuation(tokens,
                       strip_mentions=False,
                       strip_hashtags=False,
                       strict=False):

    # Use a regular expression to match and remove repeated punctuation characters
    tokens = [re.sub(r"([!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~])\1+", "", token) for token in tokens]

    # Filter punctuation tokens
    tokens = [token for token in tokens if token not in string.punctuation]

    # Remove @ symbol from left side of tokens
    if strip_mentions:
        tokens = [t.lstrip('@') for t in tokens]

    # Remove # symbol from left side of tokens
    if strip_hashtags:
        tokens = [t.lstrip('#') for t in tokens]

    return tokens

#### **Remove short tokens**

In [ ]:
def remove_short_tokens(tokens):
  # Filter the list of tokens to exclude tokens that are shorter than four letters
  filtered_tokens = [token for token in tokens if len(token) >= 4]

  # Return the filtered tokens
  return filtered_tokens

#### **Correction of spelling errors**

In [ ]:
def check_spell_errors(text, spell):

  # Correct the spelling errors in the text
  corrected_text = spell.correction(text)

  # If no correction is present user the original text
  if corrected_text == None:
     corrected_text =  text
  
  # Return the corrected text
  return corrected_text

In [ ]:
def preprocessing(documents, nlp = None, spell = None):
  corpus = []

  # process each document and append to corpus list
  for i, text in enumerate(documents):
    if i % 1000 == 0:
      print(f"{i} documents of {len(documents)}\n")
    if spell is not None:
      text = check_spell_errors(text, spell)
    tokens = tokenize(text)
    tokens = remove_links(tokens)
    tokens = remove_punctuation(tokens, strip_mentions=True, strip_hashtags=True)
    tokens = remove_numbers(tokens)
    tokens = remove_date(tokens)
    tokens = remove_short_tokens(tokens)
    tokens = remove_stopwords(tokens)
    if nlp is not None: 
      tokens = lemmatize_pt(tokens, nlp)
    #tokens = stem(tokens) 
    corpus.append(' '.join(tokens))
  return corpus

In [ ]:
# Create a SpellChecker object
spell = SpellChecker(language='pt')

nlp = spacy.load('pt_core_news_sm')

corpus = preprocessing(df.documents, nlp, spell)

In [ ]:
# Print the first document before and after pre-processing it
print(df.documents[0])
print()
print(corpus[0])

pedido cancelado sem justificativa após uma semana da compra eu estava pesquisando bastante uma nova tv para comprar e resolvi aguardar a semana do cliente, porque como ocorreu tiveram vários descontos e promoções nessa semana, sendo assim recebi descontos, cashback e cupons de varias plataformas e assim decidi efetuar a compra aquela que eu entendesse ser o melhor custo beneficio. sendo assim no dia 12 de setembro de 2022 recebi uma oferta de produto da amazon que entendi estar com um preço muito bom além de ter cachback e efetuei a compra que já aguardei até a semana do consumidor para efetuar com toda a expectativa. como estavamos anciosos e mesmo estando dentro do prazo de entrega hoje no dia 19/09 resolvi enviar uma mensagem perguntando quando o pedido seria enviado, ja que após a confirmação da compra e pagamento, passado 7 dias não tive nenhum retorno, já que complei inclusive na amazon, um site em que confio e tambem sou assinante. para a minha surpresa após o pedido de informa

## **Training a BERTopic Model**

The BERTopic algorithm has several advantages over other topic modeling algorithms. It is able to handle sparse data, it is scalable to large datasets, and it is able to learn topics that are not well-defined or are overlapping.

As our data language is portuguese we will going to set language to multilingual.

Create a new BERTopic model and train it. By default BERTopic use the paraphrase-multilingual-MiniLM-L12-v2 model for multi language documents. For others model check here [BERTopic sentence transformers](https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html#sentence-transformers)

In [ ]:
# Create a new BERTopic model using multilingual option
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True)

# Train model 
topics, probs = topic_model.fit_transform(corpus)

Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/329 [00:00<?, ?it/s]

2022-12-11 19:53:17,611 - BERTopic - Transformed documents to Embeddings
2022-12-11 19:53:57,594 - BERTopic - Reduced dimensionality
2022-12-11 19:54:01,161 - BERTopic - Clustered reduced embeddings


BERTopic works in three main steps: 


1.   Documents are first converted to numeric data. It extracts different embeddings based on the context of the word. For this, a sentence transformation model is used.
2.  Documents with similar topics are then grouped together forming clusters with similar topics. For this purpose, BERTopic uses the clustering algorithm UMAP to lower the dimensionality of the embeddings. Then the documents are clustered with the density-based algorithm HDBSCAN.
3. BERTopic extracts topics from clusters using a class-based TF-IDF score. This score gives the importance of each word in a cluster. Topics are then created based on the most important words measured by their C-TF-IDF score.

For more information check this link [BERTopic](https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6)



### **Extracting Topics**

In [ ]:
# Print the most frequent topics
freq = topic_model.get_topic_info()

# Show the top 5 most frequent topics
freq.head(5)

,Topic,Count,Name
0,-1,3887,-1_produto_entregar_pedir_compra
1,0,1455,0_amazon_prime_cobrança_cartão
2,1,664,1_entregar_atraso_prazo_data
3,2,576,2_loja_produto_entregar_magazine
4,3,433,3_bahia_casa_casas_entregar


The table above shows the five most freqeuente topics and the words present on it extract by BERTopic. -1 refers to all outliers and should be ignored.

In [ ]:
# show the most frequent topic
topic_model.get_topic(1)

[('entregar', 0.02887265484574906),
 ('atraso', 0.02654223677554765),
 ('prazo', 0.025610203572861506),
 ('data', 0.01945665111644318),
 ('chegar', 0.016913349575772108),
 ('pedir', 0.016669773675919635),
 ('hoje', 0.013946668767096972),
 ('atrasado', 0.013941323524337915),
 ('previsão', 0.013940997616260222),
 ('novembro', 0.013893448354931866)]

**Note:** BERTopic is stocastich which means that the topics might differ across runs this is mostly due to the stocastisch nature of UMAP

#### **Save topic info table as CSV**

In [ ]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/topic_info_tables/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)

# Save table as csv
freq.head(10).to_csv(path + 'topic_info_preprocessed_lemma.csv')

## **Visualization**

### **Intertopic Distance Map**

This graph shows the distance intertopic and help us understand the promixity of topics

In [ ]:
fig = topic_model.visualize_topics()
fig

#### **Save intertopic distance map**

In [ ]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/intertopic_distance_map/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "idm_preprocessed_lemma.png", format="png")

### **Visualize Topic Hierarchy**

The topics that were created can be hierarchically reduced. This visualization shows how the topics relate to one another.

In [ ]:
fig = topic_model.visualize_hierarchy(top_n_topics=30)
fig

#### **Save Hierarchical Clustering**

In [ ]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/hierarchical_clustering/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "hc_preprocessed_lemma.png", format="png")

### **Visualize Terms**

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation.

In [ ]:
fig = topic_model.visualize_barchart(top_n_topics=12)
fig

#### **Save Top Word Scores Bar Chart**

In [ ]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/top_words_scores/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "tws_preprocessed_lemma.png", format="png")

### **Visualize Topic Similarity**

This plot shows a similarity matrix by simply applying cosine similarities through those topic embeddings generate by BERTopic through both c-TF-IDF and embeddings. This matrix indicate how similar certain topics are to each other.

In [ ]:
fig = topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)
fig

#### **Save Similarity Matrix**

In [ ]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/similarity_matrix/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "sm_preprocessed_lemma.png", format="png")

### **Visualize Term Score Decline**

Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added.

In [ ]:
fig = topic_model.visualize_term_rank()
fig

#### **Save Term score decline per Topic**

In [ ]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/term_socore_decline_topic/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "tsdp_preprocessed_lemma.png", format="png")

### **Term search**

In [ ]:
# Find topics that contains blackfriday term
similar_topics, similarity = topic_model.find_topics("blackfriday", top_n=5)

# Show similar topics
similar_topics

[34, 18, 53, 54, 20]

In [ ]:
# Show a specific topic
topic_model.get_topic(34)

[('black', 0.0663716820755598),
 ('friday', 0.05866580627086462),
 ('preço', 0.03161146461277946),
 ('promoção', 0.02770985373840968),
 ('valor', 0.023885495432743015),
 ('cancelar', 0.02348781190570324),
 ('loja', 0.020913720797047883),
 ('aproveitar', 0.020875860665383664),
 ('compra', 0.019242679239039186),
 ('comprar', 0.01923376275385989)]

### **Push files to Github**

In [ ]:
import getpass

In [ ]:
password = getpass.getpass(prompt='Enter your password: ')

In [ ]:
username = getpass.getpass(prompt='Enter your username: ')

In [ ]:
email = getpass.getpass(prompt="Enter your email: ")

In [ ]:
# You can generate a token in Github -> Settings -> Developer settings
github_token = getpass.getpass(prompt="Enter your personal token: ")

In [ ]:
repo = "Topic-Modeling-Reclame-Aqui"

In [ ]:
!git config --global user.name username
!git config --global user.email email
!git config --global user.password password

In [ ]:
!git clone https://{github_token}@github.com/{username}/{repo}

In [ ]:
%cd {repo}

In [ ]:
!git add --all

In [ ]:
!git commit -a -m "Added result plots and tables files"

In [ ]:
!git remote remove origin
!git remote add origin https://{github_token}@github.com/{username}/{repo}

In [ ]:
!git remote -v

In [ ]:
!git push origin master

### **Topic Reduction**

In [ ]:
#new_topic_model = topic_model.reduce_topics(df.documents, nr_topics=40)
#new_topics = new_topic_model.topics_
#new_probs = new_topic_model.probabilities_

In [ ]:
#new_topic_model.visualize_topics()